In [0]:
from pyspark.sql.functions import current_timestamp
def add_ingestion_date(input_df):
    return input_df.withColumn("ingestion_date", current_timestamp())

In [0]:
def merge_delta_deta(input_df, db_name, table_name, folder_path, merge_condition, partition_column):
    spark.conf.set("spark.databricks.optimizer.dynamicPartitionPruning", "true")
    # by setting this, it will use the column to find the partition folder. If you not set this, the filtering condition will behave as simple hardcoded values.
    # Example in merge_condition->  "tgt.result_id = src.result_id AND tgt.race_id = src.race_id"  the race_id column is used to find the partition folder.

    from delta.tables import DeltaTable

    if(spark._jsparkSession.catalog().tableExists(f"{db_name}.{table_name}")):
        deltaTable = DeltaTable.forPath(spark, f"{folder_path}/{table_name}")
        deltaTable.alias("tgt").merge(
            input_df.alias("src"),
            merge_condition
        ) \
        .whenMatchedUpdateAll() \
        .whenNotMatchedInsertAll() \
        .execute()
    else:
        input_df.write.mode("overwrite").format("delta").partitionBy(f"{partition_column}").saveAsTable(f"{db_name}.{table_name}")